<a href="https://colab.research.google.com/github/Jinukki/KJU/blob/master/Reinforce_ch4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ch4. A2C
---
#### REINFORCE 알고리즘 단점 : 에피소드를 기다려야 하고, 그래디언트 분산이 매우 큼

#### 앞에서의 목적함수의 그래디언트

<br>

$$\nabla_\theta J(\theta)=\sum_{t=0}^T(E_{\tau \sim p_\theta(\tau)}[ \gamma^t\nabla_\theta log\pi_\theta(u_t \mid x_t)(\sum_{k=t}^T\gamma^{k-t}r(x_k, u_k))])$$

<br>

#### 여기서 가장 오른쪽 항인 <b>반환값</b>에 대해 더 명확히 알아보는 것이 목표

#### 궤적 $\tau$를 두 개의 영역으로 나누어 목적함수 그래디언트를 다르게 표현해 보자.

<br>

$$\nabla_\theta J(\theta)=\sum_{t=0}^T\int_{\tau_{x_0 :u_t}}\int_{\tau_{x_{t+1} :u_T}}( \gamma^t\nabla_\theta log\pi_\theta(u_t \mid x_t)(\sum_{k=t}^T\gamma^{k-t}r(x_k, u_k)))p_\theta(\tau_{x_0 :u_t}, \tau_{x_{t+1} :u_T})d\tau_{x_{x+1} :u_T}d\tau_{x_0 :u_t}$$
<br>
$$=\sum_{t=0}^T\int_{\tau_{x_0 :u_t}}\int_{\tau_{x_{t+1} :u_T}}( \gamma^t\nabla_\theta log\pi_\theta(u_t \mid x_t)(\sum_{k=t}^T\gamma^{k-t}r(x_k, u_k)))p_\theta(\tau_{x_{t+1} :u_T} \mid \tau_{x_0 :u_t})p_\theta(\tau_{x_0 \mid u_t})d\tau_{x_{x+1} :u_T}d\tau_{x_0 :u_t}$$
<br>
$$=\sum_{t=0}^T\int_{\tau_{x_0 :u_t}} \gamma^t\nabla_\theta log\pi_\theta(u_t \mid x_t)[\int_{\tau_{x_{t+1} :u_T}}(\sum_{k=t}^T\gamma^{k-t}r(x_k, u_k))p_\theta(\tau_{x_{t+1} :u_T} \mid \tau_{x_0 :u_t})d\tau_{x_{x+1} :u_T}]p_\theta(\tau_{x_0 \mid u_t})d\tau_{x_0 :u_t}$$

<br>

#### 마르코스 가정에 의해, 아래와 같이 정리된다.

<br>

$$\nabla_\theta J(\theta)=\sum_{t=0}^T(\int_{\tau_{x_t :u_t}} \gamma^t\nabla_\theta log\pi_\theta(u_t \mid x_t)Q^{\pi_\theta}(x_t, u_t)p_\theta(\tau_{x_t, u_t})dx_tdu_t$$

<br>

#### 위 식에 베이즈 정리를 이용하고, 후반부의 데이터 이용도를 떨어뜨리는 <b>감가율을 제거</b>하면 아래와 같다.

<br>

$$\nabla_\theta J(\theta)=\sum_{t=0}^T(\int_{(x_t,u_t)} \gamma^t\nabla_\theta log\pi_\theta(u_t \mid x_t)Q^{\pi_\theta}(x_t, u_t)\pi_\theta(u_t\mid x_t)p_\theta(x_t)dx_tdu_t)$$


<br>

$$=\sum_{t=0}^T(E_{x_t\sim p_\theta(x_t), u_t\sim \pi_\theta(u_t\mid x_t)}[\nabla_\theta log\pi_\theta(u_t \mid x_t)Q^{\pi_\theta}(x_t, u_t)])$$

<br>

#### 원래 목적함수 그래디언트와 비교하였을 때, <b>반환값 자리에 행동가치 함수</b>가 자리한 것을 알 수 있다. 

<br>

#### 즉, 에피소드가 끝날 때까지 기다려야 하는 반환값이 아닌, 이 반환값에 대한 기댓값인 행동가치를 계산함으로써 <b>에피소드가 끝날 때까지 기다릴 필요가 없다.</b>

<br>

---



#### 이제 그래디언트의 <b>분산을 줄이기 위한 방법</b>을 알아보자. 위의 마지막 목적함수 그래디언트 식에서, 행동가치 함수 대신 어떤 파라미터 $b_t$를 넣어보자.

<br>

$$\nabla_\theta J(\theta)=\sum_{t=0}^T(\int_{(x_t,u_t)} \nabla_\theta log\pi_\theta(u_t \mid x_t)b_t\pi_\theta(u_t\mid x_t)p_\theta(x_t)dx_tdu_t)$$

<br>

#### 로그 그래디언트의 변환식을 이용하면 다음과 같다.

<br>

$$=\sum_{t=0}^T(\int_{x_t}[\int_{u_t} \nabla_\theta \pi_\theta(u_t \mid x_t)b_tdu_t]p_\theta(x_t)dx_t$$

<br>

#### $b_t$가 상수이거나 $u_t$의 함수가 아니라고 하면,

<br>

$$\int_{u_t} \nabla_\theta \pi_\theta(u_t \mid x_t)b_tdu_t=b_t\nabla_\theta\int_{u_t} \pi_\theta(u_t \mid x_t)du_t=b_t\nabla_\theta(1)=0$$

<br>

#### 이므로<b>($\pi_\theta$ 확률밀도함수이므로 $u_t$에 대한 적분값이 1)</b> 목적함수 그래디언트의 값이 0이 된다. 즉, 목적함수 그래디언트 식의 행동가치 함수에서 $b_t$를 빼도 <b>기댓값은 변하지 않는다.</b>

<br>

#### 여기서 $b_t$를 <b>베이스라인(baseline)<b>이라고 한다. 분산을 줄이기 위하여 사용되므로, <b>최적값은 목적함수 그래디언트의 분산을 최소화하는 값</b>으로 선정하면 된다.

<br>

#### 일반적으로 <b>상태가치 함수 $V^{\pi_\theta}(x_t)$를 베이스라인으로 사용</b>한다. 상태가치 함수는 $u_t$의 함수가 아니므로 사용 가능하다. 즉, 목적함수 그래디언트는 다음과 같다.

<br>

$$\nabla_\theta J(\theta)=\sum_{t=0}^T(E_{x_t\sim p_\theta(x_t), u_t\sim \pi_\theta(u_t\mid x_t)}[\nabla_\theta log\pi_\theta(u_t \mid x_t)(Q^{\pi_\theta}(x_t, u_t)-V^{\pi_\theta}(x_t))])$$

<br>

$$=\sum_{t=0}^T(E_{x_t\sim p_\theta(x_t), u_t\sim \pi_\theta(u_t\mid x_t)}[\nabla_\theta log\pi_\theta(u_t \mid x_t)A^{\pi_\theta}(x_t, u_t)])$$

<br>

#### $A^{\pi_\theta}(x_t, u_t)$를 <b>어드밴티지 함수</b>라고 한다. 상태변수는 행동가치의 평균값이므로, 상태변수 $x_t$에서 선택된 행동 $u_t$가 평균에 비해 얼마나 좋은지를 평가하는 척도로 해석 가능하다.

<br>

#### 이제 목적함수 그래디언트는 어드밴티지 함수에 비례한다. 즉, <b>분산을 줄이는 문제는 어드밴티지 함수를 얼마나 정확하게 추정하느냐</b>에 따라 달라진다.

<br>

---





#### 목적함수 그래디언트는 샘플링 기법을 이용하여 근사할 수 있고, 한 개의 에피소드만 고려하면 다음과 같다.

<br>

$$\nabla_\theta J(\theta)=\sum_{t=0}^T\nabla_\theta log\pi_\theta(u_t \mid x_t)A^{\pi_\theta}(x_t, u_t))$$

<br>

#### 또한, 2장에서 <b>행동가치 함수 식은 상태가치 함수 식</b>으로 나타낼 수 있음을 확인하였고, 이에 따라 어드밴티지 함수를 다시 쓰면 아래와 같다.

<br>

$$A^{\pi_\theta}(x_t, u_t))=Q^{\pi_\theta}(x_t, u_t)-V^{\pi_\theta}(x_t)=r(x_t,u_t)+\gamma V^{\pi_\theta}(x_{t+1})-V^{\pi_\theta}(x_t) $$

<br>

#### 어드밴티지의 문제는 상태가치를 정확히 계산하는 문제로 바뀌었고, $\pi_\theta(u_t\mid x_t)$는 정책 신경망을, $\phi$는 가치신경망을 추정한다고 하면, 아래 그림처럼 두 개의 신경망이 작동하게 된다.

<br>

![A2C1](https://user-images.githubusercontent.com/52277776/84361044-090f4b80-ac06-11ea-8e49-682323e6db7d.jpg)

<br>

#### <b>정책 신경망</b>은 에이전트가 <b>어떻게 행동해야 하는지 알려주는 액터(actor) 신경망</b>이고, <b>가치 신경망</b>은 그 <b>행동을 평가하는 크리틱(critic) 신경망</b>이다. 실제로는 두 신경망이 크게 중첩되기에 다음과 같이 나타낸다.

<br>

![A2C2](https://user-images.githubusercontent.com/52277776/84361093-1c221b80-ac06-11ea-8c36-9c68ccfdb1d7.jpg)

<br>

#### 그림에서 가치함수 근사함수인 $V_\phi \approx V^{\pi_\theta}(x_t)$를 구하는 알고리즘을 고안하기 위해, 앞에서 정의했던 벨만방정식을 근사하는 방식으로 표현하면

<br>

$$V_\phi(x_t)\approx r(x_t,u_t)+\gamma V_\phi(x_{t+1})$$

<br>

#### 이고, 시간차 타깃(TD target)을 $y_i=r(x_t,u_t)+\gamma V_\phi(x_{t+1})$로 설정하면 근사 가치함수, <b>크리틱 신경망의 손실함수</b>는 아래와 같다.

<br>

$$Loss_{critic}(\phi)=\frac{1}{2}\sum_i[r(x_i,u_i)+\gamma V_\phi(x_{i+1})-V_\phi(x_i)]^2$$

<br>

#### 마찬가지로 어드밴티지 함수도 근사할 수 있고, 이에 따른 액터 신경망의 손실함수는 아래과 같다.

<br>

$$A_\phi (x_i,u_i)\approx r(x_i,u_i)+\gamma V_\phi(x_{i+1})-V_\phi(x_i)$$

<br>

$$Loss_{actor}(\theta)\approx -\sum_i(log\pi_\theta(u_i\mid x_i) A_\phi(x_i,u_i))$$

<br>

#### 따라서 최적 정책을 찾는 파라미터 $\theta$는 다음과 같이 구한다.

<br>

$$\theta \leftarrow \theta-\alpha \nabla_\theta\sum_i(log\pi_\theta(u_i\mid x_i) A_\phi(x_i,u_i))$$

<br>

#### 손실함수로 교차 엔트로피 함수를 사용함으로써 점진적으로 정책을 향상시키는 것을 확인할 수 있고,
#### 지금까지 살펴본 A2C의 알고리즘은 다음과 같다.

<br>

![A2C3](https://user-images.githubusercontent.com/52277776/84361106-1e847580-ac06-11ea-92cc-66f161acaa9b.jpg)

<br>

![A2C4](https://user-images.githubusercontent.com/52277776/84361117-2217fc80-ac06-11ea-90ab-f2f594227e94.jpg)

<br>

#### 여기에서 행동이 <b>이산공간</b>으로 표현된다면, 정책의 확률밀도함수 $\pi_\theta$를 <b>확률로 해석</b>하기 때문에 액터 신경망의 출력은 행동변수가 가질 수 있는 값의 확률이고, 액터 신경망 출력층의 뉴런의 개수도 <b>유한</b>하다.
<br>

#### 하지만 <b>연속공간</b>의 경우, 각 행동변수가 가질 수 있는 값이 <b>무한개</b>이다. 또한 액터 신경망 출력층에도 무한 개의 뉴런이 필요하다.
<br>

#### 이를 해결하기 위해 액터 신경망이 각각의 행동변수가 갖는 가우시안 분포의 <b>평균값과 분산을 출력하도록</b> 한다. 행동변수 $u$의 차원이 m이라면 액터 신경망 출력층의 뉴런 개수는 2m이다.

<br>

![A2C5](https://user-images.githubusercontent.com/52277776/84361133-27754700-ac06-11ea-9915-9a23368d9e3c.jpg)

<br>

#### 가우시안 정책 확률밀도 식의 로그형태인 가우시안 로그-정책 확률밀도 함수는 다음과 같이 평균($\mu$) 및 분산($\sigma^2$)을 이용하여 표현할 수 있다.

<br>

$$log\pi_\theta(u_t\mid x_t)=\sum_{j=i}^{m}log\pi_\theta(u_t\mid x_t)=-\sum_{j=1}^{m}[\frac{(u_{t,j}-\mu_{\theta, j}(x_t))^2}{2\sigma^2_{\theta, j}(x_t)}+\frac{1}{2}log(2\pi\sigma^2_{\theta, j}(x_t))]$$
